## Short Risks and Opportunities

This model is fairly simple, here we will look at a range of important correlations to determine whether a stock is under-or over-shorted. And under-shorted stock has the potential to decrease in price in the near future (1-2 years), and an over-shorted stock has the potential to increase in price in the near future (1-2 years), due to the risk of a short-squeeze. This model is more proprietory in nature. Daily data could have its place, but it doesn't make too much sense yet.

SHORTINT reflects positions held on the 15th business day of each month. SHORTINTME reflects position held on the last business day of the month. Hence, these variables reflect the evolution of short interest during the respective time periods. Population coverage includes the New York Stock Exchange, American Stock Exchange, and NASDAQ. This item is provided to Compustat by FT Interactive. It seems like the SEC actually carries short-interest data. You can always cik up and then ticker up to see if you can add even more connections

In [ ]:
### Note this has some problems, that short interest file might take very long to come out
### For example it has been a week, another problem is that if there is a problem in the
### Accounting and ratios feauture store then of course it would affect this feature store.

In [ ]:
# %%capture
# !pip install wrds

# import wrds
# import numpy as np
# import pandas as pd

## Ratios: https://wrds-www.wharton.upenn.edu/pages/support/research-wrds/sample-programs/wrds-sample-programs/wrds-financial-ratios-suite/
## concept: https://wrds-www.wharton.upenn.edu/pages/browse-data-concept/
## Research Applications Code: https://wrds-www.wharton.upenn.edu/pages/support/applications/

# db = wrds.Connection()

# df_short_raw = db.raw_sql("select * from comp.sec_shortint where datadate between '1960-01-01' and '2024-06-07'")
# df_short_raw.to_parquet("gs://sovai-short/short_interest/sec_short_wrds.parquet")

# df_security = db.raw_sql("select * from comp.security")
# df_security.to_parquet("gs://sovai-short/short_interest/sec_master.parquet")


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
!pip install polars scikit-learn pyarrow regex pandas gcsfs numpy fastparquet lightgbm>=4.0.0 bigframes

!pip install pyarrow>=11.0.0
!pip install lightgbm>=4.0.0
!pip install gcsfs
!pip install gcsfs
!pip install pyarrow
!pip install polars
!pip install fastparquet
!pip install pandas
!pip install s5cmd
!pip install tqdm gcsfs gspread oauth2client
!pip install boto3 duckdb
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 135.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.1 MB/s eta 0:00:00


In [4]:
# from zipfile import ZipFile
import pandas as pd
import numpy as np

# from google.colab import auth
# auth.authenticate_user()


In [ ]:
import bigframes.pandas as bpd
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

#### GRAB SHORT DATA

df_short_raw = pd.read_parquet("gs://sovai-short/short_interest/sec_short_wrds.parquet")
df_security = pd.read_parquet("gs://sovai-short/short_interest/sec_master.parquet")
df_short_raw = df_short_raw[["gvkey","iid"]].drop_duplicates()
df_short_raw = df_short_raw.merge(df_security, on=["gvkey","iid"], how="left")
df_short_raw = df_short_raw.drop_duplicates(["tic"])
df_short = pd.read_parquet("gs://sovai-short/short_interest/sec_short_wrds.parquet")
df_short_raw = df_short_raw.sort_values(["gvkey","iid"])
# Find duplicate rows based on the 'gvkey' column
# duplicate_rows = df_short_raw[df_short_raw.duplicated(subset=['gvkey'], keep=False)]

df_short = df_short.merge(df_short_raw[["gvkey","iid","tic","cusip"]], on=["gvkey","iid"], how="left")
df_short = df_short.drop(columns=["gvkey","iid"])
df_short = df_short[['tic', 'cusip'] + [col for col in df_short.columns if col not in ['tic', 'cusip']]]
df_short = df_short.drop_duplicates(["tic","datadate"], keep="first")
df_short.head()


# Note this, is just sell not short sell
project_id = 'alert-howl-193320'

# Set BigQuery DataFrames options
bpd.options.bigquery.project = project_id
bpd.options.bigquery.location = "australia-southeast2"

# Get yesterday's date
# yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# WHERE date = DATE('{yesterday}')

# Query for short_data_by_exchange table
short_data_by_exchange_query = f'''
    SELECT *
    FROM `alert-howl-193320.short_data.finra_short_interest`
'''

# Load data from the short_data table using BigFrames DataFrames
short_interest_df = bpd.read_gbq(short_data_by_exchange_query)
short_interest_df = short_interest_df.to_pandas()
short_interest_df = short_interest_df.sort_values(["symbol","settlement_date"])
short_interest_df = short_interest_df[["symbol","settlement_date","short_interest"]]
short_interest_df.columns = ["ticker","settlement_date","short_interest"]

def short_move(short_interest_df):
  # Add two weeks to the settlement_date column
  short_interest_df["date"] = pd.to_datetime(short_interest_df["settlement_date"])
  short_interest_df["date"] += pd.DateOffset(weeks=2)

  # Vectorized approach to resample dates to the next Friday
  short_interest_df["date"] = short_interest_df["date"] + pd.offsets.Week(weekday=4)
  short_interest_df["date"] = short_interest_df["date"] - pd.to_timedelta(short_interest_df["date"].dt.dayofweek, unit='d')

  # Set the index to ["ticker", "date"] for resampling
  short_interest_df = short_interest_df.set_index(["ticker", "date"])

  # Resample the DataFrame on a weekly basis ending on Friday using the new "date" column
  # short_interest_df = short_interest_df.groupby(["ticker", pd.Grouper(level="date", freq="W-FRI")]).last()

  short_interest_df = short_interest_df.groupby(level="ticker").resample("W-FRI", level="date").last()

  # Reset the index to make "ticker" and "date" regular columns
  short_interest_df = short_interest_df.reset_index()

  short_interest_df = short_interest_df.set_index(["ticker", "date"])
  short_interest_df = short_interest_df.groupby("ticker").ffill()
  short_interest_df = short_interest_df.reset_index()
  return short_interest_df

short_interest_df = short_move(short_interest_df)

df_short.head()
df_short = df_short[["tic","shortint","datadate"]]

df_short.columns = ["ticker","short_interest","settlement_date"]

df_short = df_short[["ticker","settlement_date","short_interest"]]

df_short = short_move(df_short)
df_short.shape
short_interest_df.shape
df_short = pd.concat([df_short,short_interest_df],axis=0).sort_values(["ticker","date"]).drop_duplicates(["ticker","date"], keep="first")
# import pandas as pd

# # Assuming you have already performed the resampling and forward fill operations on short_interest_df

# # Get the unique tickers from short_interest_df
# tickers_to_keep = short_interest_df["ticker"].unique()

# # Select only the tickers in df_short that are also present in short_interest_df
# df_short = df_short[df_short["ticker"].isin(tickers_to_keep)]

df_short = df_short.reset_index(drop=True)
len(df_short["ticker"].unique())
#### GRAB PRICING DATA


48390

In [ ]:
df_short.query("ticker == 'AAPL'")

,ticker,date,settlement_date,short_interest
268322,AAPL,2003-08-01,2003-07-15,14349976.0
268323,AAPL,2003-08-08,2003-07-15,14349976.0
268324,AAPL,2003-08-15,2003-07-15,14349976.0
268325,AAPL,2003-08-22,2003-07-15,14349976.0
268326,AAPL,2003-08-29,2003-07-15,14349976.0
...,...,...,...,...
269423,AAPL,2024-09-06,2024-08-15,121598771.0
269424,AAPL,2024-09-13,2024-08-15,121598771.0
269425,AAPL,2024-09-20,2024-08-30,135042504.0
269426,AAPL,2024-09-27,2024-08-30,135042504.0


In [ ]:

from google.oauth2 import service_account
from google.cloud import storage
import pandas as pd
import gcsfs
import pyarrow as pa
import pyarrow.parquet as pq


df_pricing = pd.read_parquet("gs://sovai-accounting/dataframes/prices.parq")

float_columns = ['open', 'high', 'low', 'close', 'volume', 'closeadj', 'closeunadj']
for col in float_columns:
    df_pricing[col] = df_pricing[col].astype('float32')

df_pricing['date'] = pd.to_datetime(df_pricing['date']).dt.normalize()
df_pricing.shape

df_pricing = df_pricing.drop(columns=["lastupdated"])
df_pricing = df_pricing.reset_index(drop=True)

df_pricing = df_pricing[~df_pricing["ticker"].isnull()]

print(df_pricing["ticker"].shape)
ticker_counts = df_pricing["ticker"].value_counts()

# Filter tickers with at least 30 occurrences
tickers_to_keep = ticker_counts[ticker_counts >= 30].index

# Filter the DataFrame to keep only the desired tickers
df_pricing = df_pricing[df_pricing["ticker"].isin(tickers_to_keep)]
print(df_pricing["ticker"].shape)

df_pricing = df_pricing.reset_index(drop=True)

# Perfect, so this is adjusted data.
# Open, High, Low, Close and Volume
# These fields are adjusted for stock splits and stock dividends. They are not adjusted for cash dividends or spinoffs.
df_pricing = df_pricing.drop(columns=["closeadj","closeunadj"])



(44059497,)
(44058202,)


In [ ]:
df_short.sample(100000).sort_values("date")

,ticker,date,settlement_date,short_interest
6359902,HAS,1973-02-23,1973-01-15,100.0
1673372,BDL,1973-02-23,1973-01-15,600.0
3918332,DIS,1973-02-23,1973-01-15,142416.0
12864148,SPB,1973-03-02,1973-02-15,79663.0
1839487,BIO.2,1973-03-02,1973-02-15,100.0
...,...,...,...,...
8851674,MGDDY,2024-10-04,2024-09-13,142892.0
15253523,XLY,2024-10-04,2024-09-13,4122425.0
7756013,JRSS,2024-10-04,2024-09-13,6.0
4976179,FAF,2024-10-04,2024-09-13,1475937.0


In [ ]:

import polars as pl

# Convert pandas DataFrame to polars DataFrame
df_pricing_pol = pl.from_pandas(df_pricing)

# Sort by ticker and date
df_pricing_pol = df_pricing_pol.sort(["ticker", "date"])

# Add daily features
df_pricing_pol = df_pricing_pol.with_columns(
    ((pl.col("high") - pl.col("low")) / pl.col("open")).alias("daily_volatility"),
    ((pl.col("close") - pl.col("close").shift(1)) / pl.col("close").shift(1)).alias("return_daily"),
    (pl.col("volume") / pl.col("volume").rolling_mean(window_size=20).over("ticker")).alias("relative_volume"),
)

### looks good to me, I think the problem here is nothing but ratios!!

In [ ]:
df_pricing_pol.sort("date")

ticker,date,open,high,low,close,volume,daily_volatility,return_daily,relative_volume
str,datetime[ns],f32,f32,f32,f32,f32,f32,f32,f32
"""A1""",1997-12-31 00:00:00,17.188,17.5,17.188,17.188,71800.0,0.018152,-0.881093,null
"""AAB""",1997-12-31 00:00:00,16.375,16.5,15.875,16.5,7900.0,0.038168,-0.797868,null
"""AABC""",1997-12-31 00:00:00,11.0,11.0,11.0,11.0,2000.0,0.0,-0.524324,null
"""AAC1""",1997-12-31 00:00:00,6.625,7.5,6.625,7.438,1.0664e6,0.132075,10.80635,null
"""AACB""",1997-12-31 00:00:00,11.625,12.75,11.625,12.75,10600.0,0.096774,1.582017,null
…,…,…,…,…,…,…,…,…,…
"""ZVRA""",2024-10-03 00:00:00,7.05,7.215,6.9,7.18,436757.0,0.044681,0.008427,0.350991
"""ZVSA""",2024-10-03 00:00:00,2.31,2.326,2.21,2.26,28071.0,0.050216,-0.008772,0.386361
"""ZWS""",2024-10-03 00:00:00,36.139999,36.5,36.005001,36.130001,1.461848e6,0.013697,-0.00386,0.521552


In [ ]:


# Resample to weekly level (end of week - Friday)
df_weekly = (
    df_pricing_pol
    .with_columns(
        pl.col("date").dt.truncate("1w").dt.offset_by("4d")
    )
    .group_by(["ticker", "date"])
    .agg(
        pl.first("open").alias("weekly_open"),
        pl.max("high").alias("weekly_high"),
        pl.min("low").alias("weekly_low"),
        pl.last("close").alias("weekly_close"),
        ((pl.col("volume") * pl.col("close")).sum() / pl.col("volume").sum()).alias("weekly_vwap"),
        pl.sum("volume").alias("weekly_volume"),
        pl.mean("daily_volatility").alias("avg_daily_volatility"),
        pl.mean("return_daily").alias("avg_return_daily"),
        pl.mean("relative_volume").alias("avg_relative_volume"),
    )
)


In [ ]:

# Calculate weekly return
df_weekly = df_weekly.with_columns(
    ((pl.col("weekly_close") - pl.col("weekly_open")) / pl.col("weekly_open")).alias("weekly_return")
)

# Calculate longer-term returns
df_weekly = df_weekly.with_columns(
    ((pl.col("weekly_close") / pl.col("weekly_close").shift(12)) - 1).over("ticker").alias("12week_return"),
    ((pl.col("weekly_close") / pl.col("weekly_close").shift(26)) - 1).over("ticker").alias("26week_return"),
    ((pl.col("weekly_close") / pl.col("weekly_close").shift(52)) - 1).over("ticker").alias("52week_return"),
)

# Calculate changes in volume
df_weekly = df_weekly.with_columns(
    ((pl.col("weekly_volume") / pl.col("weekly_volume").shift(12)) - 1).over("ticker").alias("12week_volume_change"),
    ((pl.col("weekly_volume") / pl.col("weekly_volume").shift(26)) - 1).over("ticker").alias("26week_volume_change"),
    ((pl.col("weekly_volume") / pl.col("weekly_volume").shift(52)) - 1).over("ticker").alias("52week_volume_change"),
)

# Calculate log returns (optional)
df_weekly = df_weekly.with_columns(
    (pl.col("weekly_close") / pl.col("weekly_close").shift(12)).log().over("ticker").alias("12week_log_return"),
    (pl.col("weekly_close") / pl.col("weekly_close").shift(26)).log().over("ticker").alias("26week_log_return"),
    (pl.col("weekly_close") / pl.col("weekly_close").shift(52)).log().over("ticker").alias("52week_log_return"),
)

window_size = 52  # Assuming weekly data, 52 weeks = 1 year
risk_free_rate = 0.02  # Assuming a 2% risk-free rate (adjust as needed)

df_weekly = df_weekly.with_columns(
    ((pl.col("weekly_return") - risk_free_rate / 52) / pl.col("avg_daily_volatility")).rolling_mean(window_size=window_size).over("ticker").alias(f"rolling_sharpe_ratio_{window_size}")
)

# Calculate rolling standard deviations for price and volume
window_sizes = [12, 26, 52]  # Adjust the window sizes as needed

for window_size in window_sizes:
    df_weekly = df_weekly.with_columns(
        pl.col("weekly_close").rolling_std(window_size=window_size).over("ticker").alias(f"rolling_price_std_{window_size}"),
        pl.col("weekly_volume").rolling_std(window_size=window_size).over("ticker").alias(f"rolling_volume_std_{window_size}")
    )

# Calculate rolling drawdowns
for window_size in window_sizes:
    df_weekly = df_weekly.with_columns(
        ((pl.col("weekly_close").rolling_max(window_size).over("ticker") - pl.col("weekly_close")) / pl.col("weekly_close").rolling_max(window_size).over("ticker")).alias(f"rolling_drawdown_{window_size}")
    )

df_weekly = df_weekly.with_columns(
    pl.col("weekly_volume").rolling_mean(window_size=8).over("ticker").alias("rolling_volume_mean_8")
)

df_weekly = df_weekly.sort(["ticker", "date"])
df_weekly = df_weekly.to_pandas()
#### GRAB RATIO DATA

df_pricing_week = pd.read_parquet("gs://sovai-accounting/processed/ratios_weekly_interpolated.parq")
df_pricing_week = df_pricing_week.merge(df_short.set_index(["ticker","date"]), left_index=True, right_index=True, how="left")
df_pricing_week = df_pricing_week.groupby("ticker").ffill()
df_pricing_week = df_pricing_week.reset_index()
df_pricing_week = df_pricing_week.dropna(subset=["settlement_date","short_interest"])
df_pricing_week = df_pricing_week.set_index(["ticker","date"])


df_weekly = df_weekly.set_index(["ticker","date"])
df_pricing_week = df_weekly.merge(df_pricing_week, left_index=True, right_index=True, how="left")
del df_weekly
df_pricing_week = df_pricing_week.dropna(subset=["short_interest","market_cap"])
df_pricing_week["number_of_shares"] = df_pricing_week["market_cap"]/df_pricing_week["weekly_close"]
df_pricing_week["short_to_float"] = df_pricing_week["short_interest"]/(df_pricing_week["number_of_shares"]*1000000)
# Convert 'short_to_float' column to numeric data type
df_pricing_week['short_to_float'] = pd.to_numeric(df_pricing_week['short_to_float'], errors='coerce')

df_pricing_week["short_to_float"] = df_pricing_week["short_to_float"].replace("NaN", np.nan)

# Drop rows where 'short_to_float' is NaN
df_pricing_week = df_pricing_week.dropna(subset=['short_to_float'])

df_pricing_week = df_pricing_week.drop(columns=["settlement_date"])

#### GRAB ACCOUNTING DATA

import pandas as pd

# Specify the GCS path to the parquet file
gcs_path = "gs://sovai-accounting/processed/accounting_weekly_interpolated.parq"

top_20_features = [
    'total_revenue',
    'net_income',
    'total_assets',
    'total_liabilities',
    'total_equity',
    'operating_income',
    'free_cash_flow',
    'ebitda',
    'current_assets',
    'current_liabilities',
    'operating_expenses',
    'net_cash_flow',
    'cost_of_revenue',
    'total_debt',
    'gross_profit',
    'net_cash_flow_operating',
    'net_cash_flow_investing',
    'net_cash_flow_financing',
    'working_capital',
    'enterprise_value'
]

# Read the specified columns from the parquet file
df_accounting = pd.read_parquet(gcs_path, columns=top_20_features)
df_pricing_week = df_pricing_week.merge(df_accounting, left_index=True, right_index=True, how="left")
#### GRAB META DATA

df_ticker = pd.read_parquet("gs://sovai-accounting/dataframes/tickers.parq")

def replace_small_cat(df, columns, thresh=0.005, term="Other"):
    for col in columns:
        frequencies = df[col].value_counts(normalize=True)
        small_categories = frequencies[frequencies < thresh].index
        df[col] = df[col].replace(small_categories, "Other")
    return df

features = ["sicindustry", "sector", "industry", "exchange", "currency", "location"]
df_ticker = replace_small_cat(df_ticker, features)
df_ticker = df_ticker[["ticker","sicindustry", "sector", "industry", "exchange", "currency", "location"]]
from sklearn.preprocessing import LabelEncoder

df_ticker[features] = df_ticker[features].apply(LabelEncoder().fit_transform)

df_ticker = df_ticker.drop_duplicates(["ticker"],keep="first")
df_ticker = df_ticker[~df_ticker["ticker"].isnull()]

df_pricing_week = pd.merge(df_pricing_week.reset_index(), df_ticker, on="ticker", how="left")
df_pricing_week['year'] = df_pricing_week['date'].dt.year
df_pricing_week['quarter'] = df_pricing_week['date'].dt.quarter
df_pricing_week = df_pricing_week.set_index(["ticker","date"])

df_pricing_week["short_to_float"] = df_pricing_week["short_to_float"].groupby('date').transform(lambda x: x.rank(pct=True))

# df_pricing_week = df_pricing_week.query("ticker == 'AAPL'")
df_pricing_week.info()

In [ ]:
import pandas as pd
import polars as pl

# Convert the DataFrame to a Polars DataFrame
df_pricing_week = pl.from_pandas(df_pricing_week.reset_index(drop=False))

In [ ]:
import pandas as pd
import polars as pl

# Calculate the percentage change over 52 weeks for the selected features
pct_change_features = [
    'short_to_float', 'total_revenue', 'net_income', 'total_assets', 'total_liabilities',
    'total_equity', 'operating_income', 'free_cash_flow', 'ebitda', 'current_assets',
    'current_liabilities', 'operating_expenses', 'net_cash_flow', 'cost_of_revenue', 'total_debt',
    'current_ratio',
    'debt_equity_ratio',
    'return_on_assets',
    'gross_profit_margin',
    'price_to_earnings',
    'asset_turnover',
    'operating_cash_flow_to_sales',
    'debt_to_capital',
    'ebitda_margin',
]

for feature in pct_change_features:
    df_pricing_week = df_pricing_week.with_columns([
        pl.col(feature).shift(52).over('ticker').pct_change().alias(f"{feature}_pct_change_52w")
    ])

# Calculate the standard deviation over 52 weeks for the selected features
std_features = [
    'short_to_float', '12week_return',  '12week_volume_change','rolling_drawdown_12'
]

for feature in std_features:
    df_pricing_week = df_pricing_week.with_columns([
        pl.col(feature).rolling_std(window_size=52).over('ticker').alias(f"{feature}_std_52w")
    ])

df_pricing_week = df_pricing_week.sort(["ticker", "date"])

# Convert the Polars DataFrame back to a Pandas DataFrame
df_pricing_week = df_pricing_week.to_pandas()
df_pricing_week = df_pricing_week.set_index(["ticker","date"])
df_pricing_week = df_pricing_week.dropna(subset=["short_to_float"])

In [ ]:
df_pricing_week.sample(100000).sort_values("date")

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Assuming df_pricing_week is already preprocessed and contains only numeric columns

# Split the data into features (X) and target (y)
X = df_pricing_week.drop(['short_to_float','short_interest'], axis=1)
y = df_pricing_week['short_to_float']

# Create LightGBM dataset
train_data = lgb.Dataset(X, label=y)

# Specify the parameters for the LightGBM model
params = {
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'verbose': 1
}

# Train the LightGBM model
model = lgb.train(params, train_data)

# Make predictions on the entire dataset
predictions = model.predict(X)

# Calculate the mean squared error
mse = mean_squared_error(y, predictions)
print(f"Mean Squared Error: {mse:.4f}")

# Add the predicted values as a new column in the DataFrame
df_pricing_week['predicted_short_to_float'] = predictions

df_pricing_week['overshorted'] = df_pricing_week['short_to_float'] - df_pricing_week['predicted_short_to_float']

## Some feature explanations too

# Take a sample of 10,000 rows from X
sample_X = X.sample(n=10000, random_state=42)

# Calculate SHAP values for the sample
lgbm_shap = model.predict(sample_X, pred_contrib=True)

# Get feature names
feature_names = sample_X.columns.tolist()

# Create a DataFrame with feature names and SHAP values
shap_df = pd.DataFrame(lgbm_shap[:, :-1], columns=feature_names)

# Calculate the mean absolute SHAP value for each feature
shap_importance = shap_df.abs().mean()

# Sort the features by their mean absolute SHAP value in descending order
shap_importance = shap_importance.sort_values(ascending=False)


In [ ]:
shap_importance

In [ ]:
df_pricing_week.head()

In [ ]:
df_pricing_week["rolling_volume_mean_8"] = df_pricing_week["rolling_volume_mean_8"].fillna(df_pricing_week["volume"])

df_pricing_week["days_to_cover"] = df_pricing_week["rolling_volume_mean_8"]/df_pricing_week["short_interest"]

# df_excerpt = df_excerpt.drop(columns=["rolling_volume_mean_8"])

df_pricing_week["number_of_shares"] = df_pricing_week["number_of_shares"]*1000000

df_pricing_week = df_pricing_week.rename(columns={"short_to_float":"short_percentage","predicted_short_to_float":"short_prediction","overshorted":"over_shorted"})

# Assuming your DataFrame is named 'df_excerpt'
df_pricing_week['over_shorted_chg'] = df_pricing_week[['over_shorted']].groupby('ticker')['over_shorted'].diff()

df_pricing_week = df_pricing_week.astype("float32")

df_pricing_week = df_pricing_week.round(3)

df_pricing_week.columns = df_pricing_week.columns.str.lower()

data_list = [
    "over_shorted",
    "over_shorted_chg",
    "short_interest",
    "number_of_shares",
    "short_percentage",
    "short_prediction",
    "days_to_cover",
    "market_cap",
    "total_revenue",
    "volume",
]

df_excerpt = df_pricing_week[data_list].copy()
df_excerpt = df_excerpt.fillna(0)

df_excerpt.to_parquet("gs://sovai-short/processed/over_shorted_frame.parquet")

df_excerpt = df_excerpt.reset_index(drop=False)

In [ ]:
df_pricing_week.to_parquet("gs://sovai-short/processed/feature_store_short.parquet")

In [5]:
import os
import subprocess
import duckdb
import polars as pl
import time
import logging

# Configure logger
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

DO_BUCKET = "sovai"
DO_ACCESS_KEY = "DO00JLGWA2N8GYY7CTKZ"
DO_SECRET_KEY = "YvDuf4HWXmrj792i5upL7EkB0I3kplMxEEyl4fpz4EE"
ENDPOINT = "nyc3.digitaloceanspaces.com"

def save_local_partition_duckdb(df: pl.DataFrame, partition_cols: list, local_dir: str) -> None:
    """
    Save a Polars DataFrame as a partitioned Parquet dataset locally using DuckDB.
    The data is sorted by the partition column(s) to group partition data together.
    """
    os.makedirs(local_dir, exist_ok=True)
    con = duckdb.connect(database=':memory:')

    partition_by = ",".join(partition_cols)
    sql = f"""
    COPY (
        SELECT * FROM df
        ORDER BY {partition_by}
    )
    TO '{local_dir}'
    (OVERWRITE, FORMAT 'parquet', PARTITION_BY '{partition_by}')
    """
    con.execute(sql)
    message = f"Local partitioning complete. Files written to: {local_dir}"
    logger.info(message)
    print(message)

def s5cmd_sync_directory(local_dir: str, do_bucket: str, remote_subdir: str,
                         do_access_key: str, do_secret_key: str, endpoint: str,
                         numworkers: int = 256, concurrency: int = 5) -> None:
    """
    Sync the local directory to DigitalOcean Spaces using s5cmd's sync command.
    Ensures that the local source directory ends with a trailing slash so that s5cmd
    syncs its contents rather than the full directory path.
    """
    if not local_dir.endswith('/'):
        local_dir += '/'

    cmd = [
        "s5cmd",
        "--endpoint-url", f"https://{endpoint}",
        "--numworkers", str(numworkers),
        "sync",
        "--size-only",
        "--concurrency", str(concurrency),
        local_dir,
        f"s3://{do_bucket}/{remote_subdir}/",
    ]
    message = f"Syncing {local_dir} to s3://{do_bucket}/{remote_subdir}/ using s5cmd sync " \
              f"with {numworkers} workers and {concurrency} concurrency."
    logger.info(message)
    print(message)

    env = os.environ.copy()
    env["AWS_ACCESS_KEY_ID"] = do_access_key
    env["AWS_SECRET_ACCESS_KEY"] = do_secret_key
    env["AWS_REGION"] = endpoint.split('.')[0]  # e.g., 'nyc3'

    try:
        result = subprocess.run(cmd, env=env, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode != 0:
            error_message = f"s5cmd sync failed:\n{result.stderr}"
            logger.error(error_message)
            print(error_message)
            raise subprocess.CalledProcessError(result.returncode, cmd, output=result.stdout, stderr=result.stderr)
        else:
            message = "s5cmd sync completed successfully."
            logger.info(message)
            print(message)
    except subprocess.CalledProcessError as e:
        error_message = f"s5cmd sync raised an error: {e}"
        logger.error(error_message)
        print(error_message)
        raise



datasets_config_normal = {
    "over_shorted": {
        "partitions": ["ticker", "date"],  # Added date partition
        "local_dirs": {
            "ticker": "/tmp/over_shorted/partitioned_data/ticker/",
            "date": "/tmp/over_shorted/partitioned_data/date/"  # Added date directory
        },
        "remote_dirs": {
            "ticker": "sovai-short/over_shorted/partitioned/ticker",
            "date": "sovai-short/over_shorted/partitioned/date"  # Added date directory
        }
    },
}

df = pl.read_parquet("gs://sovai-short/processed/over_shorted_frame.parquet")

# df = pl.from_pandas(simple_maker)
df = df.filter(pl.col("ticker").is_not_null()).select(
    "ticker", "date", pl.all().exclude(["ticker", "date"])
)

# Process each dataset sequentially.
for dataset_name, cfg in datasets_config_normal.items():
    logger.info(f"Processing dataset: {dataset_name}")
    print(f"Processing dataset: {dataset_name}")

    # Read dataset from the specified file.
    # df = pl.read_parquet(cfg["file"])


    # If the dataset is partitioned by ticker, filter out null values.
    if "ticker" in cfg["partitions"]:
        df = df.filter(pl.col("ticker").is_not_null())

    # Add partition columns. For 'ticker', just alias; for 'date' apply a transformation.
    if "ticker" in cfg["partitions"]:
        df = df.with_columns(pl.col("ticker").alias("ticker_partitioned"))
    if "date" in cfg["partitions"]:
        df = df.with_columns(
            pl.col("date")
              .dt.to_string("%G-%V-5")   # Format ISO year and week with a fixed weekday (e.g. Friday)
              .str.to_date("%G-%V-%u")    # Convert back to a Date using ISO weekday
              .alias("date_partitioned")
        )

    # Process each partition type for this dataset.
    for part in cfg["partitions"]:
        partition_col = f"{part}_partitioned"
        # If there are multiple partition columns, drop the ones not used for this run.
        current_df = df
        if len(cfg["partitions"]) > 1:
            drop_cols = [f"{p}_partitioned" for p in cfg["partitions"] if p != part]
            current_df = current_df.drop(drop_cols)

        local_dir = cfg["local_dirs"][part]
        remote_dir = cfg["remote_dirs"][part]

        logger.info(f"Partitioning {dataset_name} by '{part}'. Local dir: {local_dir}, Remote dir: {remote_dir}")
        print(f"Partitioning {dataset_name} by '{part}'. Local dir: {local_dir}, Remote dir: {remote_dir}")

        start_time = time.time()
        save_local_partition_duckdb(current_df, [partition_col], local_dir)
        elapsed = time.time() - start_time
        message = f"{dataset_name} partitioning by '{part}' took {elapsed:.2f} seconds."
        logger.info(message)
        print(message)

        start_time = time.time()
        s5cmd_sync_directory(local_dir, DO_BUCKET, remote_dir,
                            DO_ACCESS_KEY, DO_SECRET_KEY, ENDPOINT,
                            numworkers=24, concurrency=5)
        elapsed = time.time() - start_time
        message = f"s5cmd sync for {dataset_name} partitioning by '{part}' took {elapsed:.2f} seconds."
        logger.info(message)
        print(message)


Processing dataset: over_shorted
Partitioning over_shorted by 'ticker'. Local dir: /tmp/over_shorted/partitioned_data/ticker/, Remote dir: sovai-short/over_shorted/partitioned/ticker


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Local partitioning complete. Files written to: /tmp/over_shorted/partitioned_data/ticker/
over_shorted partitioning by 'ticker' took 6.53 seconds.
Syncing /tmp/over_shorted/partitioned_data/ticker/ to s3://sovai/sovai-short/over_shorted/partitioned/ticker/ using s5cmd sync with 24 workers and 5 concurrency.
s5cmd sync completed successfully.
s5cmd sync for over_shorted partitioning by 'ticker' took 67.11 seconds.
Partitioning over_shorted by 'date'. Local dir: /tmp/over_shorted/partitioned_data/date/, Remote dir: sovai-short/over_shorted/partitioned/date


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Local partitioning complete. Files written to: /tmp/over_shorted/partitioned_data/date/
over_shorted partitioning by 'date' took 2.22 seconds.
Syncing /tmp/over_shorted/partitioned_data/date/ to s3://sovai/sovai-short/over_shorted/partitioned/date/ using s5cmd sync with 24 workers and 5 concurrency.
s5cmd sync completed successfully.
s5cmd sync for over_shorted partitioning by 'date' took 9.10 seconds.


In [ ]:
# ##
# from google.cloud import storage
# import pandas as pd
# import os
# import tempfile
# from io import StringIO
# import numpy as np
# import psycopg2
# from psycopg2 import sql
# import time
# from datetime import datetime, timedelta

# # Credentials
# username = 'postgres'
# password = 'Phithae1eeja1oap'
# host = '35.192.1.147'
# port = '5432'
# database = 'altdata'
# schema = 'short'
# chunksize = 100000

# # Define TCP keepalive parameters
# keepalive_kwargs = {
#   "keepalives": 1,
#   "keepalives_idle": 1200,
#   "keepalives_interval": 10,
#   "keepalives_count": 5
# }

# # Define max retries
# MAX_RETRIES = 5

# def execute_query(cur, conn, query, data=None, retries=MAX_RETRIES):
#     print("Executing SQL:", query)  # <-- Print the SQL being executed
#     while retries > 0:
#         try:
#             cur.execute(query, data)
#             conn.commit()
#             return
#         except psycopg2.OperationalError as e:
#             if 'SSL SYSCALL' in str(e):
#                 print('Connection lost. Retrying...')
#                 retries -= 1
#                 time.sleep(5)
#                 continue
#             else:
#                 raise e
#     raise RuntimeError('Failed to execute query after multiple attempts')

# def columns_in_table(cur, table_name):
#     cur.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}';")
#     columns = [row[0] for row in cur.fetchall()]
#     return columns

# dtype_mapping = {
#     np.dtype('int32'): 'INTEGER',
#     np.dtype('int64'): 'BIGINT',
#     np.dtype('float32'): 'REAL',
#     np.dtype('float64'): 'DOUBLE PRECISION',
#     np.dtype('datetime64[ns]'): 'TIMESTAMP',
#     np.dtype('datetime64[D]'): 'DATE',
#     np.dtype('datetime64[ns]'): 'TIMESTAMP',
#     np.dtype('datetime64[ms]'): 'TIMESTAMP',
#     np.dtype('datetime64[s]'): 'TIMESTAMP',
#     np.dtype('datetime64[M]'): 'DATE',
#     np.dtype('datetime64[Y]'): 'DATE',
#     np.dtype('O'): 'TEXT',
#     'bool': 'BOOLEAN',
#     'object': 'TEXT',
#     'int32': 'INTEGER',
#     'int64': 'BIGINT',
#     'float32': 'REAL',
#     'float64': 'DOUBLE PRECISION',
#     'datetime64[ns]': 'TIMESTAMP',
#     pd.CategoricalDtype(): 'TEXT',
# }

# def save_df_to_db(df, cur, table_name):
#     temp_table_name = f"{table_name}_temp"
#     cols_with_type = ", ".join(f"{col} {dtype_mapping[df[col].dtype]}" for col in df.columns)

#     with tempfile.NamedTemporaryFile(delete=True) as temp_file:
#         df.to_csv(temp_file.name, sep='\t', index=False, header=False)
#         temp_file.seek(0)
#         cur.copy_from(temp_file, f"{temp_table_name}", null="", sep='\t', columns=df.columns)

# def send_off_data_to_psql(df_test_summary, conn, name, replace_data=False):
#     cur = conn.cursor()

#     # Setup schema and table
#     cur.execute(f"CREATE SCHEMA IF NOT EXISTS {schema};")
#     cols_with_type = ", ".join(f"{col} {dtype_mapping[df_test_summary[col].dtype]}" for col in df_test_summary.columns)

#     if replace_data:
#         cur.execute(f"DROP TABLE IF EXISTS {schema}.{name} CASCADE;")
#         cur.execute(f"CREATE TABLE {name}_temp ({cols_with_type});")
#     else:
#         cur.execute(f"CREATE TABLE IF NOT EXISTS {name}_temp ({cols_with_type});")

#     # Copy chunks
#     chunks = [df_test_summary.iloc[i:i+chunksize] for i in range(0, len(df_test_summary), chunksize)]
#     for chunk in chunks:
#         save_df_to_db(chunk, cur, name)

#     # Move and rename table
#     if replace_data:
#         cur.execute(f"DROP TABLE IF EXISTS {schema}.{name}_temp CASCADE;")
#         cur.execute(f"ALTER TABLE {name}_temp SET SCHEMA {schema};")
#         cur.execute(f"ALTER TABLE {schema}.{name}_temp RENAME TO {name};")
#     else:
#         cur.execute(f"INSERT INTO {schema}.{name} SELECT * FROM {name}_temp ON CONFLICT DO NOTHING;")
#         cur.execute(f"DROP TABLE IF EXISTS {name}_temp;")

#     conn.commit()

#     cur.close()

# # connect to your database
# conn = psycopg2.connect(
#     dbname=database,
#     user=username,
#     password=password,
#     host=host,
#     port=port,
#     **keepalive_kwargs
# )


# replace_data = True  # Set to True to replace data, False to append data

# if not df_excerpt.empty:
#     send_off_data_to_psql(df_excerpt, conn, 'over_shorted', replace_data)

In [ ]:
# df_excerpt.info()

In [ ]:
# df_excerpt.query("ticker=='MSFT'")

In [ ]:
# df_pricing_week.to_parquet("gs://sovai-short/processed/feature_store_short.parquet")

In [ ]:
# # "Volume", "Short Interest %", "Predicted SI %", "Overshorted", "Change in Overshort", "Days to Cover","Market Cap", "Total Revenue"

# df_excerpt[["short_prediction","short_percentage"]].query("ticker =='META'").plot()

# df_pricing_week[(df_pricing_week['market_cap'] > 1000) & (df_pricing_week['market_cap'] < 10000)].query("date=='2024-06-07'").sort_values("overshorted").head(20)

# df_pricing_week[df_pricing_week['market_cap']>50000].query("date=='2024-06-07'").sort_values("overshorted").head(20)

# df_pricing_week.query("date=='2024-06-07'").sort_values("overshorted")

# df_pricing_week.query("ticker=='VRAX'")["overshorted"].tail(100).plot()

# df_pricing_week.query("ticker=='TSLA'")["overshorted"].plot()

# # Take a sample of 10,000 rows from X
# sample_X = X.sample(n=10000, random_state=42)

# # Calculate SHAP values for the sample
# lgbm_shap = model.predict(sample_X, pred_contrib=True)

# # Get feature names
# feature_names = sample_X.columns.tolist()

# # Create a DataFrame with feature names and SHAP values
# shap_df = pd.DataFrame(lgbm_shap[:, :-1], columns=feature_names)

# # Calculate the mean absolute SHAP value for each feature
# shap_importance = shap_df.abs().mean()

# # Sort the features by their mean absolute SHAP value in descending order
# shap_importance = shap_importance.sort_values(ascending=False)


# shap_importance

# df_pricing_week.query("date=='2024-06-07'").sort_values("overshorted").to_csv("check.csv")



# df_pricing_week.head()

